## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-20-02-10-55 +0000,nyt,Can a University From Tennessee Help Accelerat...,https://www.nytimes.com/2025/10/19/business/va...
1,2025-10-20-02-00-00 +0000,wsj,Not long ago San Francisco was gripped by shat...,https://www.wsj.com/economy/san-francisco-is-b...
2,2025-10-20-01-56-19 +0000,nyt,"Colombia’s Leader Accuses U.S. of Murder, Prom...",https://www.nytimes.com/2025/10/19/world/ameri...
3,2025-10-20-01-52-48 +0000,bbc,Artillery shrapnel hits vehicles on California...,https://www.bbc.com/news/articles/c051p3981m4o...
4,2025-10-20-01-45-20 +0000,48hills,A ‘Tempest’ toss’d at Point Montara in sandy S...,https://48hills.org/2025/10/the-tempest-tossd-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2339/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,31
223,louvre,11
22,no,11
368,gaza,10
143,israel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
107,2025-10-19-17-28-07 +0000,nyt,Trump Posts Fake A.I. Video of Himself Flying ...,https://www.nytimes.com/2025/10/19/us/politics...,78
103,2025-10-19-17-32-48 +0000,nyt,Vance Hosts Marines Demonstration Amid ‘No Kin...,https://www.nytimes.com/2025/10/19/us/politics...,75
135,2025-10-19-12-33-49 +0000,nypost,Trump posts wild AI video showing him flying f...,https://nypost.com/2025/10/19/us-news/trump-po...,75
153,2025-10-19-10-00-00 +0000,latimes,Barabak: This Las Vegas Republican had high ho...,https://www.latimes.com/politics/story/2025-10...,74
176,2025-10-19-06-11-20 +0000,bbc,Millions turned out for anti-Trump 'No Kings' ...,https://www.bbc.com/news/articles/c93xgyp1zv4o...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
107,78,2025-10-19-17-28-07 +0000,nyt,Trump Posts Fake A.I. Video of Himself Flying ...,https://www.nytimes.com/2025/10/19/us/politics...
48,48,2025-10-19-22-12-05 +0000,nyt,"Israel Strikes Gaza and Temporarily Halts Aid,...",https://www.nytimes.com/2025/10/19/world/middl...
156,44,2025-10-19-09-57-15 +0000,nypost,"Scooter-riding thieves break into Louvre, swip...",https://nypost.com/2025/10/19/world-news/louvr...
103,44,2025-10-19-17-32-48 +0000,nyt,Vance Hosts Marines Demonstration Amid ‘No Kin...,https://www.nytimes.com/2025/10/19/us/politics...
90,44,2025-10-19-19-13-36 +0000,nypost,House Dem leader Hakeem Jeffries has awkward p...,https://nypost.com/2025/10/19/us-news/hakeem-j...
16,34,2025-10-20-00-38-24 +0000,nypost,Two dead after cargo plane skids off runway in...,https://nypost.com/2025/10/19/world-news/cargo...
41,34,2025-10-19-22-46-24 +0000,bbc,China will soon have a new Five Year Plan. Her...,https://www.bbc.com/news/articles/cjr045x425vo...
175,33,2025-10-19-06-31-02 +0000,bbc,Afghanistan pulls out of cricket series after ...,https://www.bbc.com/news/articles/c20pnz01x0eo...
128,30,2025-10-19-13-53-46 +0000,nypost,Lying ex-Rep. George Santos evokes ‘Jesus Chri...,https://nypost.com/2025/10/19/us-news/ex-rep-g...
1,27,2025-10-20-02-00-00 +0000,wsj,Not long ago San Francisco was gripped by shat...,https://www.wsj.com/economy/san-francisco-is-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
